Yael Brown<br>
October 31, 2024
## Dataset CombinerDA¶
This notebook is used combining the fraud. transaction data, price data, and normal transaction data into one dataset. del.

In [2]:
# Load libraries
import pandas as pd
import os
import math
import time
import warnings

In [3]:
# Suppress warnings in the notebook
warnings.filterwarnings("ignore")

In [4]:
# Define constants
BTC_DATA_PATH = '../data/btc_trans_data/data_raw/'

In [53]:
btc_trans_filenames = os.listdir(BTC_DATA_PATH)

btc_trans_filenames.remove('.ipynb_checkpoints')
btc_trans_filenames.sort()

print(btc_trans_filenames)

['5626370', '5626371', '5626372', '5626373', '5626374', '5626375', '5626376', '5626377', '5626378', '5626379', '5626380', '5626381', '5626382', '5626383', '5626384', '5626385', '5626386', '5626387', '5626388', '5626389', '5626390', '5626391', '5626392', '5626393', '5626394', '5626395', '5626396', '5626397', '5626398', '5626399', '5626400', '5626401', '5626402', '5626403', '5626404', '5626405', '5626406', '5626407', '5626408', '5626409', '5626410', '5626411', '5626412', '5626413', '5626414', '5626415', '5626416', '5626417', '5626418', '5626419', '5626420', '5626421', '5626422', '5626423', '5626424', '5626425', '5626426', '5626427', '5626428', '5626429', '5626430', '5626431', '5626432', '5626433', '5626434', '5626435', '5626436', '5626437', '5626438', '5626439', '5626440', '5626441', '5626442', '5626443', '5626444', '5626445', '5626446', '5626447', '5626448', '5626449', '5626450', '5626451', '5626452', '5626453', '5626454', '5626455', '5626456', '5626457', '5626458', '5626459', '5626460'

In [55]:
# Method for return year from timestamp
def get_year(row):
    ts = time.gmtime(row['timestamp'])
    return ts.tm_year

In [70]:
# Check which files are within the year range
in_year_range = []

cnt = 0
for file in btc_trans_filenames: 
    try: 
        isWithinRange = False
        df = pd.read_csv(BTC_DATA_PATH + file)
        df.sort_values(by=['timestamp'])
        
        start_year = get_year(df.iloc[0])
        end_year = get_year(df.iloc[-1])
        
        if start_year >= 2016 and end_year <= 2018:
            in_year_range.append(file)
            isWithinRange = True
        
        print(f"{cnt}/{len(btc_trans_filenames)}: {isWithinRange}    start: {start_year} end: {end_year}")
    except Exception as e:
        print(f"Error with file: {file}\n{e}")

    cnt += 1

0/289: False    start: 2009 end: 2009
1/289: False    start: 2009 end: 2009
2/289: False    start: 2009 end: 2009
3/289: False    start: 2009 end: 2009
4/289: False    start: 2009 end: 2009
5/289: False    start: 2009 end: 2009
6/289: False    start: 2009 end: 2009
7/289: False    start: 2009 end: 2009
8/289: False    start: 2009 end: 2009
9/289: False    start: 2009 end: 2009
10/289: False    start: 2009 end: 2009
11/289: False    start: 2009 end: 2009
12/289: False    start: 2010 end: 2010
13/289: False    start: 2010 end: 2010
14/289: False    start: 2010 end: 2010
15/289: False    start: 2010 end: 2010
16/289: False    start: 2010 end: 2010
17/289: False    start: 2010 end: 2010
18/289: False    start: 2010 end: 2010
19/289: False    start: 2010 end: 2010
20/289: False    start: 2010 end: 2010
21/289: False    start: 2010 end: 2010
22/289: False    start: 2010 end: 2010
23/289: False    start: 2010 end: 2010
24/289: False    start: 2011 end: 2011
25/289: False    start: 2011 end: 2

In [72]:
# Show files within range
print(in_year_range)
print(len(in_year_range))

['5626480', '5626483', '5626486', '5626489', '5626492', '5626495', '5626498', '5626500', '5626502', '5626504', '5626506', '5626508', '5626511', '5626514', '5626517', '5626520', '5626527', '5626530', '5626534', '5626536', '5626537', '5626541', '5626545', '5626549', '5626555', '5626565', '5626584', '5626587', '5626591', '5626594', '5626597', '5626600', '5626603', '5626604', '5626606', '5626609', '5626616', '5626617', '5626620', '5626667', '5635409', '5635413', '5635417']
43


In [ ]:
# Save this list because it took awhile to compute
in_year_range_save = ['5626480', '5626483', '5626486', '5626489', '5626492', '5626495', '5626498', '5626500', '5626502', '5626504', '5626506', '5626508', '5626511', '5626514', '5626517', '5626520', '5626527', '5626530', '5626534', '5626536', '5626537', '5626541', '5626545', '5626549', '5626555', '5626565', '5626584', '5626587', '5626591', '5626594', '5626597', '5626600', '5626603', '5626604', '5626606', '5626609', '5626616', '5626617', '5626620', '5626667', '5635409', '5635413', '5635417']

In [74]:
# Process and combine only files that are within range
cnt = 0 

for file in in_year_range:
    print(f"File: {file}") 
    # Read file
    df = pd.read_csv(BTC_DATA_PATH + file)
    
    # DropNA's
    df.dropna(inplace=True)

    # Create columns in dataframe for month, year, and day
    df['year'], df['month'], df['day'] = 0,0,0

    # Populate month, day, year for each row
    idx = 0
    tot_len = len(df)

    for r in range(0,tot_len): 
        try:
            ts = time.gmtime(int(df.loc[r, 'timestamp']))
            df.loc[r, 'year'] = ts.tm_year
            df.loc[r, 'month'] = ts.tm_mon
            df.loc[r, 'day'] = ts.tm_mday
            print(f"{idx}/{tot_len}/{file}")
        except Exception as e:
            df.loc[r, 'year'] = None
            df.loc[r, 'month'] = None
            df.loc[r, 'day'] = None
            print("Error for idx: ", idx)
            print(e)
        finally:
            idx += 1

    # Dropna again if there are errors processing rows
    df.dropna(inplace=True)

    # Save File and add process layer. 
    df.to_csv((BTC_DATA_PATH + f"{file}_processed.csv"), index=False, header=True, sep=',')

File: 5626480
0/24140322/5626480
1/24140322/5626480
2/24140322/5626480
3/24140322/5626480
4/24140322/5626480
5/24140322/5626480
6/24140322/5626480
7/24140322/5626480
8/24140322/5626480
9/24140322/5626480
10/24140322/5626480
11/24140322/5626480
12/24140322/5626480
13/24140322/5626480
14/24140322/5626480
15/24140322/5626480
16/24140322/5626480
17/24140322/5626480
18/24140322/5626480
19/24140322/5626480
20/24140322/5626480
21/24140322/5626480
22/24140322/5626480
23/24140322/5626480
24/24140322/5626480
25/24140322/5626480
26/24140322/5626480
27/24140322/5626480
28/24140322/5626480
29/24140322/5626480
30/24140322/5626480
31/24140322/5626480
32/24140322/5626480
33/24140322/5626480
34/24140322/5626480
35/24140322/5626480
36/24140322/5626480
37/24140322/5626480
38/24140322/5626480
39/24140322/5626480
40/24140322/5626480
41/24140322/5626480
42/24140322/5626480
43/24140322/5626480
44/24140322/5626480
45/24140322/5626480
46/24140322/5626480
47/24140322/5626480
48/24140322/5626480
49/24140322/5626

KeyboardInterrupt: 

In [ ]:
# Combine processed datasets into one dataset
btc_trans_processed_filenames = os.listdir(BTC_DATA_PATH)
btc_trans_processed_filenames.remove('.ipynb_checkpoints')

temp = []
for name in btc_trans_processed_filenames:
    if '_processed' in name:
        temp.append(name)
    else: 
        continue

btc_trans_processed_filenames = temp
del temp

In [ ]:
# Combine datasets into one dataset. 
out_df = pd.DataFrame()

cnt = 0
processed_length = len(btc_trans_processed_filenames)
for file in btc_trans_processed_filenames: 
    # Read in files
    df = pd.read_csv(BTC_DATA_PATH + file)
    
    # Concat with out_df
    out_df = pd.concat([out_df, df])

# Save output file
df.to_csv((BTC_DATA_PATH + f"btc_transactions_final.csv"), index=False, header=True, sep=',')

In [76]:
24000000 * 43

1032000000

## A lot of transactions
Since its estimated to be over a billion lines, a smaller subset from a 2016 dataset will be taken to complete EDA. 

In [82]:
df = pd.read_csv(BTC_DATA_PATH + '5626480')

df.head()

timestamp                      source_address  \
0  1451606601  1CVu8VTq7iakNBffmgJssBbN1Kje4Wz6XF   
1  1451606601  1CVu8VTq7iakNBffmgJssBbN1Kje4Wz6XF   
2  1451606601  17evnw2mbKyPB1WuFhGcYznrxJAwi1WrHx   
3  1451606601  17evnw2mbKyPB1WuFhGcYznrxJAwi1WrHx   
4  1451606601  17evnw2mbKyPB1WuFhGcYznrxJAwi1WrHx   

                  destination_address       satoshi  
0  3Qe6Zfj7Dm3fa96HXp5u8GjDARAt2MHM9V  1.000000e+09  
1   1RCgvJmbRWq77GzT5TK4mxzyzSBtHmeod  9.480760e+09  
2  3Qe6Zfj7Dm3fa96HXp5u8GjDARAt2MHM9V  1.000000e+09  
3   1RCgvJmbRWq77GzT5TK4mxzyzSBtHmeod  9.480760e+09  
4  3Qe6Zfj7Dm3fa96HXp5u8GjDARAt2MHM9V  1.000000e+09

In [83]:
# Initial shape
df.shape

(24140323, 4)

In [84]:
print("Beginning Shape: ", df.shape)

df = df.iloc[:math.floor(df.shape[0] * .0005)]

print("Ending Shape: ", df.shape)

Beginning Shape:  (24140323, 4)
Ending Shape:  (12070, 4)


In [88]:
# Verify if there are na's
df.isna().sum()

timestamp              0
source_address         0
destination_address    0
satoshi                0
dtype: int64

In [90]:
#dropna's
df.dropna(inplace=True)

# Verify if there are na's
print(df.isna().sum())

timestamp              0
source_address         0
destination_address    0
satoshi                0
dtype: int64


In [92]:
# Create columns in dataframe for month, year, and day
df['year'], df['month'], df['day'] = 0,0,0

In [94]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12070 entries, 0 to 12069
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   timestamp            12070 non-null  int64  
 1   source_address       12070 non-null  object 
 2   destination_address  12070 non-null  object 
 3   satoshi              12070 non-null  float64
 4   year                 12070 non-null  int64  
 5   month                12070 non-null  int64  
 6   day                  12070 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 660.2+ KB
None
    timestamp                      source_address  \
0  1451606601  1CVu8VTq7iakNBffmgJssBbN1Kje4Wz6XF   
1  1451606601  1CVu8VTq7iakNBffmgJssBbN1Kje4Wz6XF   
2  1451606601  17evnw2mbKyPB1WuFhGcYznrxJAwi1WrHx   
3  1451606601  17evnw2mbKyPB1WuFhGcYznrxJAwi1WrHx   
4  1451606601  17evnw2mbKyPB1WuFhGcYznrxJAwi1WrHx   

                  destination_address       satoshi  y

In [96]:
idx = 0
tot_len = len(df)

for r in range(0,tot_len): 
    try:
        ts = time.gmtime(int(df.loc[r, 'timestamp']))
        df.loc[r, 'year'] = ts.tm_year
        df.loc[r, 'month'] = ts.tm_mon
        df.loc[r, 'day'] = ts.tm_mday
        print(f"{idx}/{tot_len}")
    except Exception as e:
        df.loc[r, 'year'] = None
        df.loc[r, 'month'] = None
        df.loc[r, 'day'] = None
        print("Error for idx: ", idx)
        print(e)
    finally:
        idx += 1



0/12070
1/12070
2/12070
3/12070
4/12070
5/12070
6/12070
7/12070
8/12070
9/12070
10/12070
11/12070
12/12070
13/12070
14/12070
15/12070
16/12070
17/12070
18/12070
19/12070
20/12070
21/12070
22/12070
23/12070
24/12070
25/12070
26/12070
27/12070
28/12070
29/12070
30/12070
31/12070
32/12070
33/12070
34/12070
35/12070
36/12070
37/12070
38/12070
39/12070
40/12070
41/12070
42/12070
43/12070
44/12070
45/12070
46/12070
47/12070
48/12070
49/12070
50/12070
51/12070
52/12070
53/12070
54/12070
55/12070
56/12070
57/12070
58/12070
59/12070
60/12070
61/12070
62/12070
63/12070
64/12070
65/12070
66/12070
67/12070
68/12070
69/12070
70/12070
71/12070
72/12070
73/12070
74/12070
75/12070
76/12070
77/12070
78/12070
79/12070
80/12070
81/12070
82/12070
83/12070
84/12070
85/12070
86/12070
87/12070
88/12070
89/12070
90/12070
91/12070
92/12070
93/12070
94/12070
95/12070
96/12070
97/12070
98/12070
99/12070
100/12070
101/12070
102/12070
103/12070
104/12070
105/12070
106/12070
107/12070
108/12070
109/12070
110/12070


In [106]:
# Verify the date 
df.head(10)

timestamp                      source_address  \
0  1451606601  1CVu8VTq7iakNBffmgJssBbN1Kje4Wz6XF   
1  1451606601  1CVu8VTq7iakNBffmgJssBbN1Kje4Wz6XF   
2  1451606601  17evnw2mbKyPB1WuFhGcYznrxJAwi1WrHx   
3  1451606601  17evnw2mbKyPB1WuFhGcYznrxJAwi1WrHx   
4  1451606601  17evnw2mbKyPB1WuFhGcYznrxJAwi1WrHx   
5  1451606601  17evnw2mbKyPB1WuFhGcYznrxJAwi1WrHx   
6  1451606601  12pPD68AFg1apnUWYGEUxFSLxwJp1KXWPp   
7  1451606601  12pPD68AFg1apnUWYGEUxFSLxwJp1KXWPp   
8  1451606601  146kjf3zAC24KS4QYGrQ6qpvyz9xiSwbid   
9  1451606601  146kjf3zAC24KS4QYGrQ6qpvyz9xiSwbid   

                  destination_address       satoshi  year  month  day  
0  3Qe6Zfj7Dm3fa96HXp5u8GjDARAt2MHM9V  1.000000e+09  2016      1    1  
1   1RCgvJmbRWq77GzT5TK4mxzyzSBtHmeod  9.480760e+09  2016      1    1  
2  3Qe6Zfj7Dm3fa96HXp5u8GjDARAt2MHM9V  1.000000e+09  2016      1    1  
3   1RCgvJmbRWq77GzT5TK4mxzyzSBtHmeod  9.480760e+09  2016      1    1  
4  3Qe6Zfj7Dm3fa96HXp5u8GjDARAt2MHM9V  1.000000e+09  2016      1    1  
5   1RCgvJmbRWq77GzT5TK4mxzyzSBtHmeod  9.480760e+09  2016      1    1  
6  3Qe6Zfj7Dm3fa96HXp5u8GjDARAt2MHM9V  1.000000e+09  2016      1    1  
7   1RCgvJmbRWq77GzT5TK4mxzyzSBtHmeod  9.480760e+09  2016      1    1  
8  3Qe6Zfj7Dm3fa96HXp5u8GjDARAt2MHM9V  1.000000e+09  2016      1    1  
9   1RCgvJmbRWq77GzT5TK4mxzyzSBtHmeod  9.480760e+09  2016      1    1

In [108]:
# Verify no na's
df.isna().sum()

timestamp              0
source_address         0
destination_address    0
satoshi                0
year                   0
month                  0
day                    0
dtype: int64

In [110]:
# Write output CSV for modeling
df.to_csv((BTC_DATA_PATH + "btc_data_cleaned.csv"), index=False, header=True, sep=',')